<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/mmlu_pro/astute_rag_w_cot_oneshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain-upstage langchain-community pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: langchain-core


In [2]:
pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=a6d261ef0698f0062a277352f56a0aeacfa33ebbe4345cada7c315877a013c81
  Stored in directory: /root/.cache/pip/wheels/33/3c/79/b36253689d838af4a0539782853ac3cc38a83a6591ad570dde
Successfully built wikipedia-api


In [3]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 76.6 MB/s eta 0:00:00


In [4]:
import os
import json
from typing import List, Tuple

from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate

import wikipediaapi
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd /content/drive/MyDrive/nlp team project

/content/drive/.shortcut-targets-by-id/1gjkhHrJH8XBIbzomsGjznXFCi8ehroZK/nlp team project


In [7]:
UPSTAGE_API_KEY = "up_VYzFNHEoEJPfAwYUNp5v9n1CPnMOm"

In [8]:
df = pd.read_csv("testset.csv")

In [9]:
llm_classifier = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.1, #거의 항상 같은 결과, 안정적이게
)

In [10]:
llm_solver = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.2, #좀 더 유연한 답변을 내도록
)

In [11]:
category_index = {
    "law":        "/content/drive/MyDrive/nlp team project/code/mmlu_category/law",
    "psychology": "/content/drive/MyDrive/nlp team project/code/mmlu_category/psychology",
    "business":   "/content/drive/MyDrive/nlp team project/code/mmlu_category/business",
    "philosophy": "/content/drive/MyDrive/nlp team project/code/mmlu_category/philosophy",
    "history":    "/content/drive/MyDrive/nlp team project/code/mmlu_category/history",
}

categories = list(category_index.keys())

In [12]:

emb = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large-passage")

# 카테고리별 faiss 한번에 로드해서 캐시
vectorstore = {}
for cat, path in category_index.items():
    vectorstore[cat] = FAISS.load_local(
        folder_path=path,
        embeddings=emb,
        allow_dangerous_deserialization=True,
    )

wiki = wikipediaapi.Wikipedia(user_agent= "NLP-RAG/1.0", language = 'en')

**1st LLM**:

- return category

- return 3 keywords

In [13]:
category_prompt_template = """
You are an expert exam classifier for MMLU-Pro

There are 5 possible categories:
- law
- psychology
- business
- philosophy
- history

[Task]
Given a multiple-choice exam question (including all options),
1. Choose one best category from the list above.
2. Extract exactly 3 important keywords (one noun that consists of single word or short phrase)
   that will be useful to search textbooks and Wikipedia.
   Keywords should be as specific and accurate as possible (e.g., "consent",
   "cognitive dissonance", "Keynesian economics").

[Output format]
Return a valid JSON with the following fields:
- "category": one of ["law","psychology","business","philosophy","history"]
- "keywords": a list of exactly 3 strings

This is an output example:
{{
  "category": "psychology",
  "keywords": ["informed consent", "assent", "child counseling"]
}}

[Question]
{question}
""".strip()

category_prompt = ChatPromptTemplate.from_template(category_prompt_template)

In [14]:
def classify_and_extract_keywords(question_text: str):
    messages = category_prompt.format_messages(question=question_text)
    resp = llm_classifier.invoke(messages)
    raw = resp.content.strip()

    if not raw:
        # 완전 빈 응답이면 기본값으로 대충 처리 (죽지 않게)
        print("[WARN] Empty LLM output for category, fallback to 'history'")
        return "history", ["history", "war", "europe"]

    # 1차 시도: 전체를 JSON으로 해석
    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        # 2차 시도: 문자열 안에서 {...} 구간만 잘라서 해석
        start = raw.find("{")
        end = raw.rfind("}")
        if start == -1 or end == -1:
            print("[WARN] No JSON object found in output, fallback to 'history'")
            return "history", ["history", "war", "europe"]

        json_str = raw[start:end+1]
        try:
            data = json.loads(json_str)
        except json.JSONDecodeError as e:
            print("[WARN] JSON parse failed again:", e)
            return "history", ["history", "war", "europe"]

    category = data.get("category", "history").strip().lower()
    keywords = data.get("keywords", [])
    keywords = [str(k).strip() for k in keywords][:3]

    if not keywords:
        keywords = ["history", "war", "europe"]

    return category, keywords

In [15]:
def fetch_wiki_context(keywords, max_chars_per_page=1200):
    snippets = []
    for kw in keywords:
        try:
            page = wiki.page(kw)
            if not page.exists():
                print(f"[WARN] page for '{kw}' does not exist")
                continue
            text = page.text[:max_chars_per_page]
            snippets.append(f"[Wikipedia: {page.title}]\n{text}")
        except Exception as e:
            print(f"[WARN] wikipediaapi fetch failed for '{kw}': {e}")
            continue

    return "\n\n".join(snippets)

In [16]:
import re
import json
from typing import List, Dict, Any

import pandas as pd
from langchain_core.prompts import ChatPromptTemplate

############################################
# 0. 상수 & 기본 설정
############################################

TOP_K_TEXTBOOK = 5
TOP_K_WIKI = 3
QUESTION_COL = "prompts"



In [17]:
############################################
# 1. 질문 + 옵션(A~I) 파싱
############################################

# 1. 질문 + 옵션(A~J) 파싱
def parse_question_and_options(prompt: str):
    """
    prompt: QUESTION + (A) ~ (J) 옵션이 한 문자열에 들어있는 형태
    return: question_text(str), options(dict: 'A'~'J' -> str)
    """
    pattern = r"\(([A-J])\)\s*"
    matches = list(re.finditer(pattern, prompt))

    if not matches:
        return prompt.strip(), {}

    first = matches[0]
    question_text = prompt[:first.start()].strip()

    options: Dict[str, str] = {}
    for idx, m in enumerate(matches):
        letter = m.group(1)  # 'A'~'J'
        start = m.end()
        end = matches[idx + 1].start() if idx + 1 < len(matches) else len(prompt)
        opt_text = prompt[start:end].strip()
        options[letter] = opt_text

    return question_text, options


In [18]:
############################################
# 2. "Final Answer: X" 파서
############################################

def extract_final_answer_letter(text: str) -> str:
    """
    p_ans 출력에서 'Final Answer: X' 의 X(A~J)를 뽑기.
    """
    m = re.search(r"Final Answer:\s*([A-J])\s*$", text, flags=re.IGNORECASE | re.MULTILINE)
    if m:
        return m.group(1).upper()
    return ""

In [19]:

############################################
# 3. Memory p_gen (INTERNAL 문서 생성)
############################################

P_MEM_TMPL = """
You are a knowledgeable expert in {category}.

Task: Using ONLY your own internal knowledge (without seeing any external documents),
generate a short document that provides accurate and relevant information to help
answer the given exam question.

If you truly do not know the answer or lack enough information, explicitly state
"I don't know" rather than hallucinating facts.

Exam Question:
{question_text}

Write ONE coherent document that:
- Focuses only on concepts, definitions, and relations that are relevant to the question.
- Is self-contained and clear enough to help another model answer the question.
- Avoids unnecessary details.

Document:
""".strip()

p_mem_prompt = ChatPromptTemplate.from_template(P_MEM_TMPL)


def run_p_mem(category: str, question_text: str, llm=None) -> str:
    if llm is None:
        llm = llm_solver
    msgs = p_mem_prompt.format_messages(
        category=category,
        question_text=question_text,
    )
    resp = llm.invoke(msgs)
    return resp.content.strip()

In [20]:
############################################
# 4. Adaptive Passage Generation p_gen (각 external doc 하나씩)
############################################

P_GEN_TMPL = """
You are a careful domain expert in {category}.

Your task is to generate a short document that provides accurate and relevant
information to help answer the given exam question. You are given ONE passage
retrieved from a {source_type} source (e.g., a textbook section or a Wikipedia paragraph).

Use this passage as your primary evidence. If the information is unclear
or truly missing, explicitly state "I don't know" rather than hallucinating
new facts.

Question:
{question_text}

Retrieved passage from {source_type} (Doc {doc_id}):
{raw_passage}

Now write ONE coherent document that:
- Focuses only on information relevant to the question.
- Paraphrases and condenses the useful parts of this passage.
- Explains key concepts and relations that may help answer the question.
- Omits irrelevant details.

Document:
""".strip()

p_gen_prompt = ChatPromptTemplate.from_template(P_GEN_TMPL)


def run_p_gen_for_docs(
    category: str,
    source_type: str,          # "TEXTBOOK" or "WIKIPEDIA"
    question_text: str,
    raw_passages: List[str],   # 각 chunk 텍스트
    start_doc_idx: int = 1,
    llm=None,
) -> Tuple[str, int]:
    """
    각 raw_passage마다 p_gen을 한 번씩 호출하고,
    Astute 스타일의 [Doc N | SOURCE=...] 블록으로 이어 붙여서 리턴.
    """
    if llm is None:
        llm = llm_solver

    doc_blocks = []
    cur_idx = start_doc_idx

    for k, passage in enumerate(raw_passages, start=1):
        if not passage.strip():
            continue

        msgs = p_gen_prompt.format_messages(
            category=category,
            source_type=source_type,
            question_text=question_text,
            doc_id=k,
            raw_passage=passage,
        )
        resp = llm.invoke(msgs)
        doc_text = resp.content.strip()

        block = (
            f"[Doc {cur_idx} | SOURCE=EXTERNAL({source_type}) | ORIG_ID={source_type}_{k}]\n"
            f"{doc_text}"
        )
        doc_blocks.append(block)
        cur_idx += 1

    return "\n\n".join(doc_blocks), cur_idx

In [21]:

############################################
# 5. Retrieval helpers (textbook / wiki)
############################################

def get_textbook_passages(category: str, query: str) -> List[str]:
    """
    category별 vectorstore에서 TOP_K_TEXTBOOK개 passage 가져와 리스트로 반환.
    """
    vs = vectorstore[category]
    docs = vs.similarity_search(query, k=TOP_K_TEXTBOOK)
    return [d.page_content for d in docs]


def get_wiki_passages(keywords: List[str]) -> List[str]:
    """
    간단히 fetch_wiki_context로 받은 문자열을 하나의 passage로 처리.
    필요하면 여기서 여러 문단으로 split하여 리스트로 만들 수 있음.
    """
    try:
        raw = fetch_wiki_context(keywords)
    except Exception as e:
        print(f"[WARN] Wikipedia retrieval failed: {e}")
        return []

    raw = (raw or "").strip()
    if not raw:
        return []
    return [raw]  # 일단 1개 passage로 사용


In [22]:
############################################
# 6. Iterative Knowledge Consolidation p_con
############################################

P_CON_TMPL = """
Task: Consolidate information from both your own memorized documents (INTERNAL)
and externally retrieved documents (EXTERNAL) in response to the given exam question.

Guidelines:
* For documents that provide consistent information, cluster them together and
  summarize the key details into a single, concise document.
* For documents with conflicting information, separate them into distinct documents,
  ensuring each document captures the unique perspective or data.
* Exclude any information that is irrelevant to the question.
* For each NEW document you create, clearly indicate:
  - Whether the source was INTERNAL(MEMORY) or EXTERNAL(TEXTBOOK/WIKIPEDIA).
  - The original document numbers that contributed to it (e.g., FROM=1,3,4).

Initial Context (numbered documents):
{context_init}

Last Context (may be empty):
{last_context}

Question:
{question_text}

Now produce your New Context as a list of numbered documents in the following style:

[Doc 1 | SOURCE=EXTERNAL(TEXTBOOK) | FROM=2,4]
...document text...

[Doc 2 | SOURCE=INTERNAL(MEMORY) | FROM=1]
...document text...

New Context:
""".strip()

p_con_prompt = ChatPromptTemplate.from_template(P_CON_TMPL)


def run_p_con(
    question_text: str,
    context_init: str,
    last_context: str = "",
    llm=None,
) -> str:
    if llm is None:
        try:
            llm = llm_solar_pro
        except NameError:
            llm = llm_solver

    msgs = p_con_prompt.format_messages(
        question_text=question_text,
        context_init=context_init,
        last_context=last_context,
    )
    resp = llm.invoke(msgs)
    return resp.content.strip()


In [23]:
############################################
# 7. Knowledge Consolidation + Answer Finalization p_ans
############################################

P_ANS_TMPL = """
You are an expert exam solver for MMLU-Pro.

Task: Answer the given multiple-choice question using the consolidated information
from both your own memorized knowledge (INTERNAL) and the externally retrieved documents.

Below is an example of how you should reason and format your answer.

[EXAMPLE]

Initial Context:
(INTERNAL) In classical conditioning, a neutral stimulus is paired with an unconditioned stimulus.

[Consolidated Context]:
(EXTERNAL-TEXTBOOK) Classical conditioning was first systematically described by Ivan Pavlov.
(EXTERNAL-WIKIPEDIA) Classical conditioning associates a neutral stimulus with a reflex response.

[QUESTION]
In classical conditioning, the neutral stimulus should:
(A) Immediately elicit a response before any pairing.
(B) Be repeatedly paired with the unconditioned stimulus.
(C) Be presented only after extinction has occurred.
(D) Remain completely unrelated to the unconditioned stimulus.

Reasoning:
Step 1 – Consolidate information again (mentally)
- INTERNAL says a neutral stimulus is paired with an unconditioned stimulus.
- TEXTBOOK says Pavlov described this process of pairing.
- WIKIPEDIA says the neutral stimulus becomes associated with a reflex response.
- All sources agree that repeated pairing is essential.

Step 2 – Extract essential clues from the QUESTION
- Clue 1: The question asks about the role of the neutral stimulus in classical conditioning.
- Clue 2: The neutral stimulus does NOT initially elicit the response.
- Clue 3: The neutral stimulus must be paired with the unconditioned stimulus.

Step 3 – Compare each option (A–D)
- (A) is contradicted: the neutral stimulus does not elicit a response before pairing.
- (B) matches the clues: it exactly describes repeated pairing with the unconditioned stimulus.
- (C) is irrelevant: extinction is not mentioned in the clues or context.
- (D) contradicts the idea of association and pairing.

Step 4 – Select the Final Answer
Option (B) is fully consistent with all clues and evidence,
whereas the other options are contradicted or unsupported.

Final Answer: B

Now follow the same procedure for the NEW problem below.

Step 1 – Consolidate information again (mentally)
* Read the Initial Context and the Consolidated Context.
* Identify the key facts and note whether each comes mainly from INTERNAL(MEMORY)
  or EXTERNAL(TEXTBOOK/WIKIPEDIA).
* Group them into a small set of evidence clusters.
* Exclude any information that is irrelevant to the question.
* If INTERNAL and EXTERNAL disagree, explain which seems more reliable and why.
* If two EXTERNAL sources disagree, treat TEXTBOOK as more reliable than WIKIPEDIA.

Step 2 – Extract essential clues from the QUESTION
* Look only at the QUESTION (stem + options) and extract 3–5 essential clues
  (core concepts, constraints, or relationships).
* If these QUESTION clues conflict with any context, ALWAYS trust the QUESTION clues.

Step 3 – Compare each option (A–J)
For each option:
* Explain briefly (1–3 sentences) whether it is supported or contradicted by the
  evidence clusters and QUESTION clues.
* If there is not enough information, say so but still compare it to other options.

Step 4 – Select the Final Answer
* Choose the SINGLE best option (A–J) that fits the QUESTION clues and the
  strongest evidence.
* If the context is incomplete, still choose the MOST reasonable option that does
  not contradict the evidence.
* Provide a short explanation (3–6 sentences) summarizing why this option is better
  than the others.

Important formatting rules:
- At the VERY END, output the final answer in the exact format:
  Final Answer: X
- Replace X with a single capital letter between A and J.
- Do NOT wrap the line in Markdown (no **, no backticks).
- Do NOT put the letter in parentheses or quotes.
- Do NOT add any text after the 'Final Answer: X' line.
- This line MUST appear only once.

Initial Context:
{context_init}

[Consolidated Context]:
{context_con}

[QUESTION]
{question_with_options}

Now reason step by step according to the steps above, then give the final answer.
""".strip()

p_ans_prompt = ChatPromptTemplate.from_template(P_ANS_TMPL)


def run_p_ans(
    context_init: str,
    context_con: str,
    full_question: str,
    llm=None,
) -> Dict[str, Any]:
    if llm is None:
        try:
            llm = llm_solar_pro
        except NameError:
            llm = llm_solver

    msgs = p_ans_prompt.format_messages(
        context_init=context_init,
        context_con=context_con,
        question_with_options=full_question,
    )
    resp = llm.invoke(msgs)
    raw = resp.content.strip()
    letter = extract_final_answer_letter(raw)
    return {"final_answer": letter, "raw_reasoning": raw}

In [24]:
############################################
# 8. 한 문제 풀기 (Astute-RAG 스타일)
############################################

def solve_mmlu_astute_style(
    full_prompt: str,
    use_wiki: bool = True,
) -> Dict[str, Any]:
    """
    1) question/option 파싱
    2) category, keywords 분류
    3) memory doc + textbook/wiki passages -> p_gen
    4) p_con으로 consolidated context 생성
    5) p_ans로 최종 답 선택
    """
    # 1) 질문/선지 분리
    question_text, options = parse_question_and_options(full_prompt)

    # 2) category / keywords
    category, keywords = classify_and_extract_keywords(question_text)
    if category not in vectorstore:
        print(f"[WARN] Unknown category '{category}', fallback to 'business'")
        category = "business"

    # 3) Retrieval
    retrieval_query = question_text + " " + " ".join(options.values())
    tb_passages = get_textbook_passages(category, retrieval_query)
    wiki_passages = get_wiki_passages(keywords) if use_wiki else []

    # 4) p_mem: INTERNAL 문서
    doc_blocks = []
    next_doc_idx = 1

    mem_doc_text = run_p_mem(category, question_text)
    if mem_doc_text.strip():
        doc_blocks.append(
            f"[Doc {next_doc_idx} | SOURCE=INTERNAL(MEMORY)]\n{mem_doc_text}"
        )
        next_doc_idx += 1

    # 5) p_gen: TEXTBOOK docs
    tb_block, next_doc_idx = run_p_gen_for_docs(
        category=category,
        source_type="TEXTBOOK",
        question_text=question_text,
        raw_passages=tb_passages,
        start_doc_idx=next_doc_idx,
        llm=llm_solver,
    )
    if tb_block.strip():
        doc_blocks.append(tb_block)

    # 6) p_gen: WIKIPEDIA docs
    if use_wiki and wiki_passages:
        wiki_block, next_doc_idx = run_p_gen_for_docs(
            category=category,
            source_type="WIKIPEDIA",
            question_text=question_text,
            raw_passages=wiki_passages,
            start_doc_idx=next_doc_idx,
            llm=llm_solver,
        )
        if wiki_block.strip():
            doc_blocks.append(wiki_block)

    context_init = "\n\n".join(doc_blocks)

    # 7) p_con: consolidation (한 번만)
    context_con = run_p_con(
        question_text=question_text,
        context_init=context_init,
        last_context="",
    )

    # 8) p_ans: 최종 정답
    ans = run_p_ans(
        context_init=context_init,
        context_con=context_con,
        full_question=full_prompt,
    )

    return {
        "final_answer": ans["final_answer"],
        "category": category,
        "keywords": keywords,
        "context_init": context_init,
        "context_con": context_con,
        "raw_reasoning": ans["raw_reasoning"],
    }

50개

In [28]:
import numpy as np
import pandas as pd

df = pd.read_csv("/content/testset_100 (1).csv")

# 🔹 100개 중 랜덤 50개 인덱스만 선택
sample_indices = df.sample(n=50, random_state=42).index   # random_state 바꾸면 샘플 바뀜

for i in sample_indices:
    row = df.loc[i]
    full_prompt = row[QUESTION_COL]

    print(f"[{i}] Solving: {str(full_prompt)[:60]}...")

    out = solve_mmlu_astute_style(
        full_prompt=full_prompt,
        use_wiki=True,
    )

    final_ans = out["final_answer"]
    df.loc[i, "astute_answer"] = final_ans
    df.loc[i, "pred_category"] = out["category"]

    kws = out["keywords"]
    df.loc[i, "kw1"] = kws[0] if len(kws) > 0 else ""
    df.loc[i, "kw2"] = kws[1] if len(kws) > 1 else ""
    df.loc[i, "kw3"] = kws[2] if len(kws) > 2 else ""

    # reasoning / context 저장
    df.loc[i, "cot_full"] = out["raw_reasoning"]
    df.loc[i, "context_init"] = out["context_init"]
    df.loc[i, "context_con"] = out["context_con"]

out_name = "astute-rag-1121-50sample.csv"
df.to_csv(out_name, index=False)
print(f"저장 완료: {out_name}")

[83] Solving: QUESTION240) Daniel receives at 6.5% commission on all sales...
[WARN] page for 'percentage calculation' does not exist
[53] Solving: QUESTION10878) Which branch of Judaism founded by Zacharias ...
[WARN] page for 'Positive-Historical Judaism' does not exist
[70] Solving: QUESTION2694) Which of the following is not an available too...
[WARN] page for 'suicide risk' does not exist
[WARN] page for 'assessment tools' does not exist
[45] Solving: QUESTION1485) Hume's attack on natural law is founded on his...
[44] Solving: QUESTION10881) "The minor premise must affirm the antecedent...
[39] Solving: QUESTION11239) Zhuangzi describes a state as ziran, which me...
[22] Solving: QUESTION11192) Select the best translation into predicate lo...
[80] Solving: QUESTION4715) This question refers to the following informat...
[WARN] page for 'Colonel Cresap' does not exist
[10] Solving: QUESTION2449) Dr. Ryan is a psychotherapist in a small town....
[WARN] page for 'therapist-client bou

In [29]:
df_sample = df.loc[sample_indices].copy()
df_sample.to_csv("astute-rag-1121-50only.csv", index=False)

기존 testset.csv

In [25]:
import pandas as pd

df = pd.read_csv("testset.csv")[26:]  # 질문, 선택지 다 합쳐진 컬럼 가정
QUESTION_COL = "prompts"

results = []
for i, row in df.iterrows():
    q = row[QUESTION_COL]
    print(f"[{i}] Solving: {q[:60]}...")
    out = solve_mmlu_astute_style(q)
    results.append(out["final_answer"])
    df.loc[i, "pred_category"] = out["category"]
    df.loc[i, "kw1"] = out["keywords"][0] if len(out["keywords"]) > 0 else ""
    df.loc[i, "kw2"] = out["keywords"][1] if len(out["keywords"]) > 1 else ""
    df.loc[i, "kw3"] = out["keywords"][2] if len(out["keywords"]) > 2 else ""
    df.loc[i, "rag_cot_answer"] = out["final_answer"]
    df.loc[i, "cot_full"] = out["raw_reasoning"]

df.to_csv("baseline-1120.csv", index=False)
print("저장 완료:baseline-1120.csv")

[26] Solving: QUESTION27) A man is at home in his apartment, alone, late a...
[27] Solving: QUESTION28) What do Homo sapiens and Australopithecus afaren...
[28] Solving: QUESTION29)This question refers to the following information...
[WARN] page for 'frontier garrisons' does not exist
[29] Solving: QUESTION30)Homo erectus differed from Homo habilis in which ...
[WARN] page for 'evolutionary differences' does not exist
[30] Solving: QUESTION31)During the manic phase of a bipolar disorder, ind...
[WARN] page for 'manic phase' does not exist
[31] Solving: QUESTION32) This question refers to the following informatio...
[WARN] page for 'Germany's responsibility' does not exist
[32] Solving: QUESTION33) You receive a phone call from Hermann H., age 28...
[WARN] page for 'ethical psychologist' does not exist
[33] Solving: QUESTION34) During the second stage of Kohlberg’s preconvent...
[WARN] page for 'preconventional level' does not exist
[34] Solving: QUESTION35)  In satisfying Kant's Humani

In [26]:
import pandas as pd

# 1. 파일 경로
GT_PATH = "testset.csv"          # 정답 파일
PRED_PATH = "baseline-1120.csv"       # 출력 파일

# 2. 컬럼 이름
GT_COL = "answers"               # testset.csv에서 정답 컬럼
PRED_COL = "rag_cot_answer"      # baseline.csv에서 예측 컬럼

# 3. csv 로드
gt_df = pd.read_csv(GT_PATH)[26:]      # 26번 문제부터라고 가정
pred_df = pd.read_csv(PRED_PATH)

# 4. Series 추출
gt = gt_df[GT_COL].astype(str)
pred = pred_df[PRED_COL].astype(str)

# 5. 정규화 함수
def normalize_choice(x: str) -> str:
    x = x.strip().upper()
    for ch in x:
        if "A" <= ch <= "Z":
            return ch
    return x

# 인덱스 리셋이 포인트
gt_norm = gt.apply(normalize_choice).reset_index(drop=True)
pred_norm = pred.apply(normalize_choice).reset_index(drop=True)

print(len(gt_norm), len(pred_norm))  # 둘 다 25 나오는지 체크 한번 해보고

# 6. 정확도 계산
correct = (gt_norm == pred_norm)
accuracy = correct.mean()

print(f"총 문제 수: {len(gt_norm)}")
print(f"정답 개수: {correct.sum()}")
print(f"정확도: {accuracy * 100:.2f}%")

24 24
총 문제 수: 24
정답 개수: 19
정확도: 79.17%
